In [ ]:
# NBA Timeout Analysis Report
# Analyzing the Effectiveness of Timeouts in Disrupting Opponent Momentum in NBA Games

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import warnings

# Suppress warnings for clean output
warnings.filterwarnings('ignore')

# Set the path to figures folder
FIGURES_PATH = "DSA-210-PROJECT/dsa project/outputs/figures/"

# Set style for matplotlib
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

# Title and Introduction
print("# NBA Timeout Analysis: Do Timeouts Stop Opponent Momentum?")
print("## An Analysis of NBA Timeout Effectiveness in Disrupting Scoring Runs")
print("\n")
print("### Introduction")
print("""
In basketball, coaches often call timeouts when the opposing team is on a scoring run, commonly referred to as "stopping the bleeding." This analysis explores whether this coaching strategy is effective by examining NBA timeout data from five seasons (1999-00, 2004-05, 2010-11, 2016-17, 2022-23).

**Research Question:** Do timeouts effectively disrupt opponent momentum during scoring runs?

**Hypothesis:** When the opponent team makes a scoring run of 6-0 or better and a timeout is called, the opponent's offensive efficiency decreases after the timeout compared to before.

**Methodology:** We analyzed play-by-play data from NBA games, identifying instances where:
1. A team went on a scoring run of 6+ points to 0
2. The opposing team called a timeout
3. We compared the offensive efficiency of the scoring team before and after the timeout

In this notebook, we'll examine the results of this analysis, looking at overall effectiveness, trends by season, scoring run sizes, quarters, and team-specific patterns.
""")

# Load the data
print("### Loading and Preparing Data")
print("First, let's load our timeout analysis results dataset...")

# Assuming the CSV is in outputs folder
try:
    data_file = "outputs/timeout_analysis_results.csv"
    df = pd.read_csv(data_file)
    
    # Filter out 1996-97 season (if present)
    if 'season' in df.columns:
        df = df[df['season'] != '1996-97']
        
    print(f"Successfully loaded data with {len(df)} timeout records after filtering.")
    print("\nFirst few rows of the data:")
    
    # Show a sample of the data
    print(df.head().to_string())
    
    # Data description
    print("\nSummary statistics:")
    print(df.describe().to_string())
    
    # Key column explanations
    print("\n**Key Column Explanations:**")
    print("- **effective**: Whether the timeout reduced opponent offensive efficiency (TRUE) or not (FALSE)")
    print("- **efficiency_change**: Difference in opponent offensive efficiency (points per possession) after the timeout compared to before")
    print("- **pre_timeout_oe/post_timeout_oe**: Opponent offensive efficiency before/after timeout")
    print("- **run_terminated**: Whether the opponent's scoring run was stopped after the timeout")
    print("- **run_points**: Size of the opponent's scoring run before the timeout")
    
except FileNotFoundError:
    print(f"Error: Could not find the data file {data_file}.")
    # Create placeholder data for demonstration
    print("Creating demonstration data for visualization purposes...")
    
    # Sample data based on the expected results
    np.random.seed(42)
    n_samples = 2470
    
    seasons = ['1999-00', '2004-05', '2010-11', '2016-17', '2022-23']
    teams = ['ORL', 'NOP', 'DET', 'TOR', 'BOS', 'MEM', 'OKC', 'MIN', 'POR', 'PHX', 'CLE', 'ATL', 'SAS', 'LAC', 'UTA']
    quarters = ['Q1', 'Q2', 'Q3', 'Q4', 'OT1']
    run_sizes = ['6-7', '8-9', '10-11', '12-14', '15-19', '20+']
    
    season_dist = np.random.choice(seasons, n_samples, p=[0.25, 0.17, 0.18, 0.2, 0.2])
    team_dist = np.random.choice(teams, n_samples)
    quarter_dist = np.random.choice(quarters, n_samples, p=[0.21, 0.27, 0.25, 0.26, 0.01])
    run_size_dist = np.random.choice(run_sizes, n_samples, p=[0.96, 0.035, 0.0048, 0.0001, 0.0001, 0.0])
    
    pre_timeout_oe = np.random.normal(18, 5, n_samples)
    noise = np.random.normal(-3.655, 15.798, n_samples)
    post_timeout_oe = pre_timeout_oe + noise
    effective = post_timeout_oe < pre_timeout_oe
    
    df = pd.DataFrame({
        'season': season_dist,
        'opponent_abbr': team_dist,
        'quarter': quarter_dist,
        'run_size_bin': run_size_dist,
        'pre_timeout_oe': pre_timeout_oe,
        'post_timeout_oe': post_timeout_oe,
        'efficiency_change': post_timeout_oe - pre_timeout_oe,
        'effective': effective,
        'run_terminated': np.random.choice([True, False], n_samples, p=[0.574, 0.426]),
        'pre_timeout_fg_pct': np.random.uniform(0.4, 0.7, n_samples),
        'post_timeout_fg_pct': np.random.uniform(0.3, 0.55, n_samples),
        'pre_timeout_ts': np.random.uniform(0.5, 0.75, n_samples),
        'post_timeout_ts': np.random.uniform(0.4, 0.65, n_samples)
    })
    
    print("Created demonstration data with 2,470 timeout records.")

# 1. Overall Effectiveness Analysis
print("\n### 1. Overall Timeout Effectiveness")
print("""
Let's start by examining the overall effectiveness of timeouts in disrupting opponent momentum. A timeout is considered "effective" if the opponent's offensive efficiency (points per possession) decreased after the timeout compared to before the timeout.
""")
effective_timeouts = df['effective'].sum()
total_timeouts = len(df)
effectiveness_rate = effective_timeouts / total_timeouts
avg_efficiency_change = df['efficiency_change'].mean()

print(f"**Total timeouts analyzed:** {total_timeouts}")
print(f"**Effective timeouts:** {effective_timeouts} ({effectiveness_rate*100:.1f}%)")
print(f"**Average change in opponent efficiency:** {avg_efficiency_change:.3f} points per possession")

t_stat, p_value = stats.ttest_1samp(df['efficiency_change'], 0)
print(f"t-statistic: {t_stat:.3f}, p-value: {p_value:.3e}")
